# SageMaker Model Deployment Examples

This notebook demonstrates how to deploy a model using different SageMaker deployment options: Real-Time Endpoint, Serverless Inference, Batch Transform, Asynchronous Inference, and Multi-Model Endpoint.

## Data Preparation and Training

In [25]:
# Step 1: Setup
import sagemaker
from sagemaker import get_execution_role
import boto3
import pandas as pd
from sklearn.model_selection import train_test_split 

sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = boto3.Session().region_name

# S3 bucket for storing data
bucket = 'sagemaker-ml-28573'
prefix = 'deployment'
output_path = f's3://{bucket}/{prefix}/output'

# Load the dataset
file_path = 'Employee.csv'  # Replace with your actual file path in S3 if needed
employee_df = pd.read_csv(file_path)
employee_df.head()



# Step 2: Data Preparation
# Convert categorical columns to numeric
employee_df['Education'] = employee_df['Education'].astype('category').cat.codes
employee_df['City'] = employee_df['City'].astype('category').cat.codes
employee_df['Gender'] = employee_df['Gender'].astype('category').cat.codes
employee_df['EverBenched'] = employee_df['EverBenched'].map({'Yes': 1, 'No': 0})

# Drop rows with NaN values in the target column
employee_df.dropna(subset=['LeaveOrNot'])

# Convert target column to numeric if needed
employee_df['LeaveOrNot'] = employee_df['LeaveOrNot'].astype(int)

# Ensure no missing values in feature columns
employee_df = employee_df.dropna()

# Verify all columns are numeric
print(employee_df.dtypes)

# Define features and target
feature_columns = [
    'Education', 'JoiningYear', 'City', 'PaymentTier', 'Age',
    'Gender', 'EverBenched', 'ExperienceInCurrentDomain'
]
target_column = 'LeaveOrNot'

employee_df = employee_df[[target_column] + feature_columns]

train_df, test_df = train_test_split(employee_df, test_size=0.2, random_state=42)

# Display the transformed dataset
employee_df.head()






Education                     int8
JoiningYear                  int64
City                          int8
PaymentTier                  int64
Age                          int64
Gender                        int8
EverBenched                  int64
ExperienceInCurrentDomain    int64
LeaveOrNot                   int64
dtype: object


,LeaveOrNot,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain
0,0,0,2017,0,3,34,1,0,0
1,1,0,2013,2,1,28,0,0,3
2,0,0,2014,1,3,38,0,0,2
3,1,1,2016,0,3,27,1,0,5
4,1,1,2017,2,3,24,1,1,2


In [26]:
# Initialize S3 client
s3 = boto3.client('s3')


# Save the data locally first
train_file = 'train.csv'
validation_file = 'validation.csv'
train_df.to_csv(train_file, index=False)
test_df.to_csv(validation_file, index=False)

# Upload the data to S3
s3.upload_file(train_file, bucket, f'{prefix}/train/{train_file}')
s3.upload_file(validation_file, bucket, f'{prefix}/validation/{validation_file}')

print(f"Training data uploaded to s3://{bucket}/{prefix}/train/{train_file}")
print(f"Validation data uploaded to s3://{bucket}/{prefix}/validation/{validation_file}")

# Setup XGBoost Estimator
xgboost_container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name, "1.3-1")
hyperparameters = {
    "max_depth":"5",
    "eta":"0.2",
    "gamma":"40",
    "min_child_weight":"6",
    "subsample":"0.7",
    "objective":"binary:logistic",
    "num_round":"50"
}

output_path = f's3://{bucket}/{prefix}/output'

estimator = Estimator(
    image_uri=xgboost_container, 
    hyperparameters=hyperparameters,
    role=sagemaker.get_execution_role(),
    instance_count=1, 
    instance_type='ml.m5.xlarge', 
    volume_size=5,  # 5 GB 
    output_path=output_path
)

# Define the data type and paths to the training and validation datasets
content_type = "csv"
train_input = TrainingInput(f"s3://{bucket}/{prefix}/train/{train_file}", content_type=content_type)
validation_input = TrainingInput(f"s3://{bucket}/{prefix}/validation/{validation_file}", content_type=content_type)

# Execute the XGBoost training job
estimator.fit({'train': train_input, 'validation': validation_input})

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-09-03-12-15-08-548


Training data uploaded to s3://sagemaker-ml-28573/deployment/train/train.csv
Validation data uploaded to s3://sagemaker-ml-28573/deployment/validation/validation.csv
2024-09-03 12:15:10 Starting - Starting the training job...
2024-09-03 12:15:23 Starting - Preparing the instances for training...
2024-09-03 12:16:11 Downloading - Downloading the training image......
2024-09-03 12:17:02 Training - Training image download completed. Training in progress.
2024-09-03 12:17:02 Uploading - Uploading generated training model[2024-09-03 12:16:57.415 ip-10-2-209-70.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-09-03 12:16:57.437 ip-10-2-209-70.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-09-03:12:16:57:INFO] Imported framework sagemaker_xgboost_container.training
[2024-09-03:12:16:57:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2024-09-03:12:16:57:INFO] No GPUs detect

## Deploy as Real-Time Endpoint

In [27]:

# Deploy the model as a real-time endpoint
predictor = estimator.deploy(
    initial_instance_count=1,  # Number of instances to deploy
    instance_type='ml.m5.xlarge',  # Instance type for the endpoint
    endpoint_name='employee-attrition-predictor'  # Name of the endpoint
)


INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-09-03-12-20-13-053
INFO:sagemaker:Creating endpoint-config with name employee-attrition-predictor
INFO:sagemaker:Creating endpoint with name employee-attrition-predictor


------!

In [30]:

# Optionally, configure the predictor for the specific input and output formats
predictor.serializer = sagemaker.serializers.CSVSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()



# Example input data (in the same format as your training data)

# Assuming 'LeaveOrNot' is the target column
features_df = test_df.drop(columns=['LeaveOrNot'])

# Convert the features DataFrame to CSV format
test_csv = features_df.to_csv(index=False, header=False).strip()

test_data = test_df[feature_columns].head(20)  # Select the first row of test data for prediction
test_data_csv = test_data.to_csv(index=False, header=False).strip()  # Convert to CSV format

In [31]:


# Invoke the endpoint
response = predictor.predict(test_data_csv)
print(response)  # The response will be in JSON format, containing the predicted label




{'predictions': [{'score': 0.20303820073604584}, {'score': 0.20303820073604584}, {'score': 0.3774380087852478}, {'score': 0.5340070128440857}, {'score': 0.1691509336233139}, {'score': 0.37122291326522827}, {'score': 0.6700358986854553}, {'score': 0.8091390132904053}, {'score': 0.1691509336233139}, {'score': 0.8091390132904053}, {'score': 0.1691509336233139}, {'score': 0.20303820073604584}, {'score': 0.9251188635826111}, {'score': 0.1691509336233139}, {'score': 0.35528063774108887}, {'score': 0.8091390132904053}, {'score': 0.1633274257183075}, {'score': 0.38120347261428833}, {'score': 0.36576730012893677}, {'score': 0.1691509336233139}]}


In [ ]:
# Example input data (in the same format as your training data)
test_data = test_df[feature_columns].head(1)  # Select the first row of test data for prediction
test_data_csv = test_data.to_csv(index=False, header=False).strip()  # Convert to CSV format

# Invoke the endpoint
response = predictor.predict(test_data_csv)
print(response)  # The response will be in JSON format, containing the predicted label

In [32]:
# Delete the endpoint when no longer needed
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: employee-attrition-predictor
INFO:sagemaker:Deleting endpoint with name: employee-attrition-predictor


In [ ]:
from sagemaker.model import Model

# Specify the S3 path to the pre-trained model artifact
model_artifact = "s3://your-path/output/model.tar.gz"

# Retrieve the container image for the framework (e.g., XGBoost)
container = sagemaker.image_uris.retrieve(framework="xgboost", region=boto3.Session().region_name, version="1.3-1")

# Create the model object using the S3 path
model = Model(
    image_uri=container,
    model_data=model_artifact,
    role=sagemaker.get_execution_role()
)

# Deploy the model as a real-time endpoint
predictor = model.deploy(
    initial_instance_count=1,  # Number of instances
    instance_type='ml.m5.xlarge',  # Instance type
    endpoint_name='employee-attrition-predictor'  # Name of the endpoint
)

# Optionally, configure the predictor for the specific input and output formats
predictor.serializer = sagemaker.serializers.CSVSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

# Invoke the endpoint for predictions
response = predictor.predict(test_data_csv)
print(response)



In [36]:
import boto3

# Initialize the SageMaker runtime client
runtime_client = boto3.client('sagemaker-runtime')

# Specify the endpoint name
endpoint_name = 'employee-attrition-predictor'

# Prepare your input data (same format as before)
test_data = test_df[feature_columns].head(25)  # Select the first row of test data for prediction
test_data_csv = test_data.to_csv(index=False, header=False).strip()  # Convert to CSV format

# Invoke the endpoint directly using the runtime client
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="text/csv",  # Specify the content type
    Body=test_data_csv  # The input data as a CSV string
)

# Parse the response
result = response['Body'].read().decode('utf-8')
print(result)


0.20906561613082886
0.1683102250099182
0.38488397002220154
0.4326663315296173
0.15928953886032104
0.46319764852523804
0.6720890998840332
0.9404475688934326
0.15928953886032104
0.9504724144935608
0.12999935448169708
0.20906561613082886
0.9239230155944824
0.12999935448169708
0.2676047682762146
0.9504724144935608
0.15928953886032104
0.42358148097991943
0.47207778692245483
0.12999935448169708
0.15928953886032104
0.5771342515945435
0.709308922290802
0.1683102250099182
0.3065555989742279



In [ ]:
# Delete the endpoint when no longer needed
predictor.delete_endpoint()


## Deploy as Serverless Inference Endpoint

In [41]:
import boto3
from sagemaker.model import Model
from sagemaker.serverless import ServerlessInferenceConfig
from sagemaker import get_execution_role


# Create a Model object using the trained estimator
model = Model(
    image_uri=xgboost_container,
    model_data=estimator.model_data,
    role=sagemaker.get_execution_role()
)

# Alternative
# Create the model object using the S3 path
# model = Model(
#     image_uri=container,
#     model_data=model_artifact,
#     role=get_execution_role()
# )

# Define the Serverless Inference configuration
serverless_inference_config = ServerlessInferenceConfig(
    memory_size_in_mb=2048,  # Allocate memory
    max_concurrency=5  # Max concurrent invocations
)

# Deploy the model as a serverless endpoint
serverless_predictor = model.deploy(
    serverless_inference_config=serverless_inference_config,
    endpoint_name='employee-attrition-serverless-1'
)





INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-09-03-12-42-28-507
INFO:sagemaker:Creating endpoint-config with name employee-attrition-serverless-1
INFO:sagemaker:Creating endpoint with name employee-attrition-serverless-1


-----!

In [43]:
import boto3

# Initialize the SageMaker runtime client
runtime = boto3.client("sagemaker-runtime")

# Specify your serverless endpoint name
endpoint_name = "employee-attrition-serverless-1"  # Replace with your actual endpoint name

# Specify the content type
content_type = "text/csv"

# Use the CSV string from test_df as the payload
payload = test_data_csv

# Invoke the endpoint
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=payload
)

# Read and decode the response
result = response['Body'].read().decode('utf-8').splitlines()

# Print the predictions
print(result)


['0.20303820073604584', '0.20303820073604584', '0.3774380087852478', '0.5340070128440857', '0.1691509336233139', '0.37122291326522827', '0.6700358986854553', '0.8091390132904053', '0.1691509336233139', '0.8091390132904053', '0.1691509336233139', '0.20303820073604584', '0.9251188635826111', '0.1691509336233139', '0.35528063774108887', '0.8091390132904053', '0.1633274257183075', '0.38120347261428833', '0.36576730012893677', '0.1691509336233139', '0.1691509336233139', '0.40060654282569885', '0.7384953498840332', '0.20303820073604584', '0.3876122534275055']


In [ ]:
print(len(result))  # Should match len(test_df)

print(test_data_csv)

In [ ]:
# Example input data (in the same format as your training data)
test_data = test_df[feature_columns].head(23)  # Select the first row of test data for prediction
test_data_csv = test_data.to_csv(index=False, header=False).strip()  # Convert to CSV format

## Deploy using Batch Transform

In [115]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.transformer import Transformer  # Import the Transformer class

# S3 bucket for storing data
bucket = 'sagemaker-ml-28573'
prefix = 'deployment'

output_data = f's3://{bucket}/{prefix}/output/'


# Specify the S3 path to the pre-trained model artifact
model_artifact = "s3://sagemaker-ml-28573/demo-built-in-algorithm/output/sagemaker-xgboost-2024-08-29-13-30-18-037/output/model.tar.gz"


# Retrieve the container image for the framework (e.g., XGBoost)
container = sagemaker.image_uris.retrieve(framework="xgboost", region=boto3.Session().region_name, version="1.3-1")


# Create the model object using the S3 path
model = Model(
    image_uri=container,
    model_data=model_artifact,
    role=sagemaker.get_execution_role()
)


# Create a transformer object
transformer = model.transformer(
    instance_count=1,
    instance_type='ml.m5.large',  # Choose an instance type
    output_path=output_data,
    strategy='MultiRecord',  # Strategy for processing records (SingleRecord or MultiRecord)
    assemble_with='Line',  # How to join results, e.g., 'Line' to join with newlines
    accept='text/csv'  # Output format
)



# Start the transform job
transformer.transform(
    data=input_data,  # Input data in S3
    content_type='text/csv',  # Input format
    split_type='Line'  # How the input data is split (e.g., by line)
)

# Wait for the job to finish
transformer.wait()



# The results will be available in the S3 output path specified


INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-09-03-15-58-23-641
INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-09-03-15-58-24-464


ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateTransformJob operation: The account-level service limit 'ml.m5.large for transform job usage' is 0 Instances, with current utilization of 0 Instances and a request delta of 1 Instances. Please use AWS Service Quotas to request an increase for this quota. If AWS Service Quotas is not available, contact AWS support to request an increase for this quota.

## Deploy as Asynchronous Inference Endpoint

In [ ]:

from sagemaker.async_inference import AsyncInferenceConfig

# Deploy the model as an asynchronous endpoint
async_predictor = estimator.deploy(
    initial_instance_count=1,  # Number of instances
    instance_type='ml.m5.xlarge',  # Instance type
    async_inference_config=AsyncInferenceConfig(
        output_path=f's3://{bucket}/async-output',  # S3 path to store output
        max_concurrent_invocations_per_instance=2
    ),
    endpoint_name='employee-attrition-async-1'
)


In [116]:
# Remove the label column and the header
features_only = test_df.iloc[:, 1:]  # Exclude the first column (label)
test_data_csv = features_only.to_csv(index=False, header=False).strip()

# Save to CSV without header
csv_file_path = 'validation_no_label.csv'
features_only.to_csv(csv_file_path, index=False, header=False)

# Upload the prepared CSV to S3
s3 = boto3.client('s3')
s3.upload_file(csv_file_path, bucket, f'{prefix}/validation/validation_no_label.csv')

# Update the input location for the asynchronous request
input_location = f's3://{bucket}/{prefix}/validation/validation_no_label.csv'

In [117]:
import boto3

# Create a low-level client representing Amazon SageMaker Runtime
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)



# The name of the asynchronous endpoint that you have deployed

endpoint_name = 'employee-attrition-async-1'

# Invoke the asynchronous endpoint with your input data location
response = sagemaker_runtime.invoke_endpoint_async(
    EndpointName=endpoint_name, 
    InputLocation=input_location,
    InvocationTimeoutSeconds=1600,  # Set timeout to allow sufficient time for processing
    ContentType='text/csv'
)

# Response contains metadata about the request, not the prediction itself.
# The actual predictions will be saved to the specified S3 output path.
print("Asynchronous inference request sent. Check S3 for results.")


Asynchronous inference request sent. Check S3 for results.


In [110]:
print(f's3://{bucket}/{prefix}/validation/validation_no_label.csv')

s3://sagemaker-ml-28573/deployment/validation/validation_no_label.csv


In [111]:
print(input_location)

s3://sagemaker-ml-28573/deployment/validation/validation_no_label.csv


In [ ]:
# Clean up the asynchronous endpoint
async_predictor.delete_endpoint()

## Deploy as Multi-Model Endpoint

In [ ]:
from sagemaker import Model
from sagemaker.multidatamodel import MultiDataModel

# S3 path to your model artifacts
model_artifact_path = 's3://sagemaker-ml-28573/deployment/output/sagemaker-xgboost-2024-09-03-12-15-08-548/output/'

# Define the container image (for example, XGBoost)
container_image = sagemaker.image_uris.retrieve(framework="xgboost", region=boto3.Session().region_name, version="1.3-1")

# Create a Model object (this includes the image and role information)
model = Model(
    image_uri=container_image,
    role=sagemaker.get_execution_role()
)

# Create the MultiDataModel
mme = MultiDataModel(
    name="multi-model-endpoint",
    model_data_prefix=model_artifact_path,  # S3 path where models are stored
    model=model,  # Pass the Model object that defines the container image
    sagemaker_session=sagemaker.Session()
)

# Deploy the Multi-Model Endpoint
predictor = mme.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    endpoint_name="multi-model-endpoint"
)


In [138]:
import boto3
from botocore.config import Config

# Initialize the SageMaker Runtime client with a retry strategy
config = Config(
    read_timeout=70,
    retries={
        'max_attempts': 2  # Adjust this value as needed (up to 5 for a max timeout of 360s)
    }
)
runtime_sagemaker_client = boto3.client('sagemaker-runtime', config=config)

# Define the endpoint name and the specific model to target
endpoint_name = "multi-model-endpoint"
target_model = "model.tar.gz"

# Example CSV input data

# Invoke the endpoint
response = runtime_sagemaker_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="text/csv",
    TargetModel=target_model,
    Body=test_data_csv
)

# Print the response from the model
print(response['Body'].read().decode('utf-8'))


[0.20303820073604584, 0.20303820073604584, 0.3774380087852478, 0.5340070128440857, 0.1691509336233139, 0.37122291326522827, 0.6700358986854553, 0.8091390132904053, 0.1691509336233139, 0.8091390132904053, 0.1691509336233139, 0.20303820073604584, 0.9251188635826111, 0.1691509336233139, 0.35528063774108887, 0.8091390132904053, 0.1633274257183075, 0.38120347261428833, 0.36576730012893677, 0.1691509336233139, 0.1691509336233139, 0.40060654282569885, 0.7384953498840332, 0.20303820073604584, 0.3876122534275055, 0.1691509336233139, 0.1691509336233139, 0.1633274257183075, 0.1633274257183075, 0.1633274257183075, 0.1691509336233139, 0.9031620621681213, 0.3774380087852478, 0.1691509336233139, 0.32054612040519714, 0.1691509336233139, 0.37556877732276917, 0.1691509336233139, 0.20303820073604584, 0.1691509336233139, 0.90727698802948, 0.1691509336233139, 0.8091390132904053, 0.1691509336233139, 0.1691509336233139, 0.1691509336233139, 0.1691509336233139, 0.6700358986854553, 0.9093160629272461, 0.1691509

In [137]:
print(test_data_csv)

0,2016,0,3,24,0,0,2
0,2013,0,3,26,0,0,4
0,2017,1,2,25,0,0,3
1,2015,2,2,28,0,0,2
0,2012,0,3,33,1,0,1
1,2012,1,3,24,0,0,2
0,2016,2,3,27,0,0,5
0,2015,2,2,25,0,1,3
0,2015,1,3,28,1,0,1
0,2015,2,2,27,0,0,5
0,2013,0,3,27,1,0,5
0,2017,0,3,24,0,0,2
0,2018,2,2,39,0,0,4
0,2012,0,3,27,1,1,5
1,2017,1,2,34,1,1,2
0,2015,2,2,26,0,0,4
0,2014,2,3,24,1,0,2
2,2016,1,2,28,1,0,3
1,2012,2,3,27,1,0,5
0,2016,0,3,40,1,0,5
0,2012,0,3,28,1,0,0
0,2015,2,2,25,1,0,3
0,2017,2,2,33,0,0,2
0,2015,0,3,27,0,0,5
0,2012,0,1,27,0,0,5
0,2015,0,3,26,1,0,4
0,2013,0,3,27,1,0,5
0,2014,2,3,25,1,0,3
0,2013,2,3,27,1,0,5
0,2014,2,3,41,1,0,2
0,2014,0,3,25,1,0,3
1,2018,2,3,39,1,0,2
0,2017,1,2,28,0,0,2
0,2012,0,3,25,1,0,3
1,2017,0,3,40,0,0,2
0,2017,0,3,39,1,0,2
1,2013,1,3,26,1,0,4
0,2015,0,3,27,1,1,5
0,2013,1,3,29,0,0,1
0,2014,0,3,28,1,0,2
0,2018,0,3,26,1,0,4
0,2016,0,3,27,1,0,5
0,2015,2,2,27,0,1,5
0,2014,1,3,25,1,0,3
0,2012,0,3,27,1,0,5
0,2015,0,3,25,1,0,3
0,2013,0,3,38,1,0,0
0,2016,2,3,24,0,0,2
0,2018,0,3,27,0,1,5
0,2016,0,3,37,1,0,2


In [ ]:
# Clean up the multi-model endpoint
multi_model_predictor.delete_endpoint()